In [1]:
import dask.dataframe as dd
import numpy as np
import pickle

v = "4.02"
id_path = r"..\\cls\\eos_l2v\\%s\\" %v
corpus_path = r"..\\corpus\\survival_set\\sentences\\test\\part*"

#### Description
This notebook is used to aggregate the group data for the *Mortality* model evaluation

In [4]:
with open(id_path + "id.npy", "rb") as f:
    idx = np.load(f)
    
data = dd.read_parquet(corpus_path)
data = data[~data["AFTER_THRESHOLD"]]
data["LENGTH"] = data["SENTENCE"].apply(lambda x: len(x.split()), meta=(None, "int"))
data["HAS_HEALTH"] = data["LENGTH"].apply(lambda x: x<=3, meta=(None, "int"))

In [5]:
seqlen = data.groupby("PERSON_ID")["LENGTH"].sum().compute()
with open(id_path + "seqlen.pkl", "wb") as f:
    pickle.dump(seqlen.to_dict(), f)

In [10]:
has_health =  data.groupby("PERSON_ID")["HAS_HEALTH"].sum().compute()
with open(id_path + "has_health.pkl", "wb") as f:
    pickle.dump(has_health.to_dict(), f)